In [1]:
require 'nn'
require 'hdf5'

# Loading data

In [31]:
-- Train format is (number of Ngrams, Ngram_size + 1) with last
-- col the count of the N_gram of the line

-- Validation format is (number of words to predict, 50 + Ngrams_size -1)
-- where the 50 columns stands for the 50 words possibilities in the prediction,
-- the next col stands for the current context (goal is to predict the Nth word)

myFile = hdf5.open('1-grams.hdf5','r')
data = myFile:all()
train_1 = data['train']
myFile:close()

myFile = hdf5.open('2-grams.hdf5','r')
data = myFile:all()
train_2 = data['train']
validation_2 = data['valid']
validation_output = data['valid_output']
myFile:close()

myFile = hdf5.open('3-grams.hdf5','r')
data = myFile:all()
train_3 = data['train']
validation_3 = data['valid']
myFile:close()

myFile = hdf5.open('4-grams.hdf5','r')
data = myFile:all()
train_4 = data['train']
validation_4 = data['valid']
myFile:close()

myFile = hdf5.open('5-grams.hdf5','r')
data = myFile:all()
train_5 = data['train']
test_5 = data['test']
validation_5 = data['valid']
myFile:close()

myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
train_6 = data['train']
test_6 = data['test']
validation_6 = data['valid']
myFile:close()

# Maximum Likelihood Estimation

In [3]:
-- Second version: reversing the order in F_c_w

function build_context_count(count_tensor)
    -- Ngram count (depend on w and context)
    -- {'index1-...-indexN-1': {'indexN' : count}}
    local F_c_w = {}
    -- F_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count all words in c}
    local F_c = {}
    -- N_c dict (independent of w, only context based)
    -- {index1-...-indexN-1 : count unique type of words in c}
    local N_c = {}

    local N = count_tensor:size(1)
    local M = count_tensor:size(2)

    for i=1, N do
        indexN = count_tensor[{i,M-1}]
        
        -- build the key index1-...-indexN-1
        indexes = tostring(count_tensor[{i,1}])
        for j=2, M - 2 do
            indexes = indexes .. '-' .. tostring(count_tensor[{i,j}])
        end
        
        -- Filling F_c_w
        if F_c_w[indexes] == nil then
            F_c_w[indexes] = {[indexN] = count_tensor[{i, M}]}
        else
            F_c_w[indexes][indexN] = count_tensor[{i, M}]
        end
        
        -- Updating F_c and F_c
        if F_c[indexes] == nil then
            F_c[indexes] = count_tensor[{i, M}]
            N_c[indexes] = 1
        else
            F_c[indexes] = count_tensor[{i, M}] + F_c[indexes]
            N_c[indexes] = 1 + N_c[indexes]
        end
    end
    
    return F_c_w, F_c, N_c
end

In [4]:
-- Prediction with the MLE (with Laplace smoothing)

function mle_proba(data, F_c_w, alpha)
    local N = data:size(1)
    local M = data:size(2)
    -- Output format: distribution predicted for each N word along the
    -- 50 possibilities
    local distribution = torch.DoubleTensor(N, 50)

    for i=1, N do
        -- build the key index1-...-indexN-1
        indexes = tostring(data[{i,51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i,j}])
        end
        
        -- Look up in the dictionnary for the 50 possible ngrams asked
        for j=1, 50 do
            indexN = data[{i,j}]
            if F_c_w[indexN] == nil or F_c_w[indexN][indexes] == nil then
                distribution[{i,j}] = alpha
            else
                distribution[{i,j}] = F_c_w[indexN][indexes] + alpha
            end
        end
        -- Debug: case where no n-gram were found (only when alpha=0.)
        if distribution:narrow(1,i,1):sum(2)[{1,1}] == 0 then
            -- Select the first one (most common)
            distribution[{i,1}] = 1
        end
    end
    -- normalization (ie we do the MLE given only the 50 possibilities)
    distribution:cdiv(torch.expand(distribution:sum(2), N, 50))
    
    return distribution
end

In [5]:
-- Loading train of the gram_size N
function get_train(N)
    local filename = N .. '-grams.hdf5'
    print(filename)
    myFile = hdf5.open(filename,'r')
    train = myFile:all()['train']
    myFile:close()
    return train
end

In [6]:
-- Prediction with the MLE (with Laplace smoothing) and fix on never seen context

function mle_proba_2(data, N, alpha)
    local N_data = data:size(1)
    local M = data:size(2)
    -- Output format: distribution predicted for each N_data word along the
    -- 50 possibilities
    local distribution = torch.DoubleTensor(N_data, 50)
    local gram_size
    
    -- Building the count matrix for each ngram size lower than N,
    -- if need to reduce the context in case of unseen new context
    local F_c_w_table = {}
    for i=1,N do
        train = get_train(i)
        F_c_w, F_c, N_c = build_context_count_2(train)
        F_c_w_table[i] = F_c_w
    end

    for i=1, N_data do
        -- Initialize the Ngram_size
        gram_size = N
        -- build the key index1-...-indexN-1
        indexes = tostring(data[{i,51}])
        for j=52, M do
            indexes = indexes .. '-' .. tostring(data[{i,j}])
        end
        -- case context never seen before:
        -- we look for the reduced context (ie index2-...-indexN-1)
        if F_c_w_table[N][indexes] == nil then
            -- Look for previous context
            for k = N-1,1,-1 do
                -- update gram_size
                gram_size = k
                -- Building index in reduced context
                indexes_split = indexes:split("-")
                indexes = ''
                for i= 1 + N - gram_size, N-1 do
                    indexes = indexes .. indexes_split[i]
                end
                -- look if current context seen
                if F_c_w_table[gram_size][indexes] ~= nil then
                    break
                end
            end 
        end

        -- Look up in the dictionnary for the 50 possible ngrams asked
        for j=1, 50 do
            indexN = data[{i,j}]
            if gram_size == 1 then
                -- Format of indexes is always string
                indexes = tostring(indexN)
            end
            -- case word never seen in that context: smoothing only
            if F_c_w_table[gram_size][indexes][indexN] == nil then
                distribution[{i,j}] = alpha
            else
                distribution[{i,j}] = F_c_w_table[gram_size][indexes][indexN] + alpha
            end
        end

        -- case uniform prediction (because no word present in current context)
        if distribution:narrow(1,i,1):sum(2)[{1,1}] == 50 * alpha then
            -- TODO: try with previous context
            -- Select the first one (most common)
            distribution[{i,1}] = 1 + alpha
            --print('here', i, j, gram_size, indexes, indexN)
        end
    end
    -- normalization (ie we do the MLE given only the 50 possibilities)
    distribution:cdiv(torch.expand(distribution:sum(2), N_data, 50))
    
    return distribution
end

In [7]:
function perplexity(distribution, true_words)
    -- exp of the average of the cross entropy of the true word for each line
    -- true words (N_words to predict, one hot true value among 50)
    local perp = 0
    local N = true_words:size(1)
    for i = 1,N do
        mm,aa = true_words[i]:max(1)
        perp = perp + math.log(distribution[{i, aa[1]}])
    end
    perp = math.exp(- perp/N)
    return perp
end

In [32]:
-- Test of new mle version
distribution_mle_2_new = mle_proba_2(validation_2, 2, 1)

1-grams.hdf5	


2-grams.hdf5	


In [34]:
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2_new, validation_output))

Result on alpha smoothing 2grams	8.8674198826233	


In [88]:
F_c_w, F_c, N_c = build_context_count(train_2)
distribution_mle_2 = mle_proba(validation_2, F_c_w, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2, validation_output))

Result on alpha smoothing 2grams	8.8680603647573	


In [38]:
-- Test of new mle version
distribution_mle_3_new = mle_proba_2(validation_3, 3, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_3_new, validation_output))

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


Result on alpha smoothing 2grams	17.059968522337	


In [10]:
F_c_w_table = {}
for i=1,3 do
    train = get_train(i)
    F_c_w, F_c, N_c = build_context_count_2(train)
    F_c_w_table[i] = F_c_w
end

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


In [11]:
validation_2:narrow(1, 3, 1)

Columns 1 to 13
  500   305  8425  5919  4090  4981  8611  8244  4804  2009  9910   471     6

Columns 14 to 26
 9904  1198   514  1061  5049  6289  8585  1697  4648  3676  7371  1867  4857

Columns 27 to 39
 3924  6939  8431  3086  4288  3138  4486  3949  3730  8607  7310  9582  9873

Columns 40 to 51
 6426  7482  1215  6676  9535  1417  6268  7276  1861  4232  7448  3884
[torch.LongTensor of size 1x51]



In [92]:
F_c_w_table[1]['6594']

{
  6594 : 10
}


In [7]:
F_c_w, F_c, N_c = build_context_count(train_3)
distribution_mle_3 = mle_proba(validation_3, F_c_w, 1)

In [8]:
F_c_w, F_c, N_c = build_context_count(train_4)
distribution_mle_4 = mle_proba(validation_4, F_c_w, 1)

In [11]:
-- Results on the validation set
print('Result on alpha smoothing 2grams', perplexity(distribution_mle_2, validation_output))
print('Result on alpha smoothing 3grams', perplexity(distribution_mle_3, validation_output))
print('Result on alpha smoothing 4grams', perplexity(distribution_mle_4, validation_output))

Result on alpha smoothing 2grams	8.8680603647573	


Result on alpha smoothing 3grams	21.513973503713	


Result on alpha smoothing 4grams	29.942969799245	


# Witten Bell Model

In [25]:
function compute_wb_line(N, entry, F_c_w_table, F_c_table, N_c_table, alpha)
    -- Return vector (50) predicting the distribution from entry
    -- N represent the Ngram size used in the prediction so context is N-1 gram
    -- alpha is only used for the MLE without any context
    local prediction = torch.zeros(50)
    local indexN
    -- case where computation only on the prior
    if N == 1 then
        for j=1,50 do
            indexN = entry[{1, j}]
            -- Corner case when prediction on words not on the dict (case for <s>)
            if F_c_w_table[1][tostring(indexN)] == nil then
                prediction[j] = 0
            else
                prediction[j] = F_c_w_table[1][tostring(indexN)][indexN] + alpha
            end
        end
        -- Normalizing
        return prediction:div(prediction:sum(1)[1])
    else
        -- Compute the MLE for current N
        -- context (at least with one element)
        local indexes = tostring(entry[{1, entry:size(2)}])
        for j=entry:size(2) - 1, entry:size(2) - 1 - (N-3), -1 do
            indexes = tostring(entry[{1, j}]) .. '-' .. indexes
        end
        -- check if context is unseen, otherwise go to next context
        if F_c_w_table[N][indexes] == nil then
            --print('unseen context')
            return compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha)
        end
        -- Compute MLE for each word
        for j=1,50 do
            indexN = entry[{1, j}]
            if F_c_w_table[N][indexes][indexN] ~= nil then
                prediction[j] = F_c_w_table[N][indexes][indexN]
            end
        end
        
        -- Check that MLE predicted at least one words, otherwise go to next context
        if prediction:sum(1)[1] == 0 then
            --print('unseen words')
            return compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha)
        end
        
        -- Combining with next context
        prediction:add(compute_wb_line(N-1, entry, F_c_w_table, F_c_table, N_c_table, alpha):mul(N_c_table[N][indexes]))
        -- Normalization
        -- TODO: We normalize as we apply on a reduced dataest (50 words)
        prediction:div(prediction:sum(1)[1])
        return prediction
    end
end

In [20]:
-- Witten Bell: new version, computation done at once line by line
--
-- p_wb(w|c) = (F_c_w + N_c_. * p_wb(w|c'))/(N_c_. + F_c_.)
function distribution_proba_WB(N, data, alpha)
    local N_data = data:size(1)
    local M = data:size(2)

    -- Building the count matrix for each ngram size lower than N.
    local F_c_w_table = {}
    local F_c_table = {}
    local N_c_table = {}
    for i=1,N do
        train = get_train(i)
        F_c_w_table[i], F_c_table[i], N_c_table[i] = build_context_count(train)
    end

    -- Vector initialisation
    local distribution = torch.zeros(N_data, 50)
    for i=1,N_data do
        -- Compute witten bell for the whole line i
        distribution:narrow(1, i, 1):copy(compute_wb_line(N, data:narrow(1,i,1), F_c_w_table, F_c_table, N_c_table, alpha))
    end
    return distribution
end

In [269]:
t = compute_wb_line(2, validation_3:narrow(1,6,1), F_c_w_table, F_c_table, N_c_table, 1)

unseen words	


In [11]:
distribution_2 = distribution_proba_WB(2, validation_2, 1)

1-grams.hdf5	


2-grams.hdf5	


In [21]:
distribution_1 = distribution_proba_WB(1, validation_2, 1)
print('Result on alpha smoothing 1grams', perplexity(distribution_1, validation_output))
distribution_2 = distribution_proba_WB(2, validation_2, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_2, validation_output))
distribution_3 = distribution_proba_WB(3, validation_3, 1)
print('Result on alpha smoothing 3grams', perplexity(distribution_3, validation_output))
distribution_4 = distribution_proba_WB(4, validation_4, 1)
print('Result on alpha smoothing 4grams', perplexity(distribution_4, validation_output))

1-grams.hdf5	


Result on alpha smoothing 1grams	6.0029978880708	
1-grams.hdf5	


2-grams.hdf5	


Result on alpha smoothing 2grams	5.8434260885191	
1-grams.hdf5	


2-grams.hdf5	


...

Result on alpha smoothing 3grams	6.8723895034594	
1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


Result on alpha smoothing 4grams	7.4900378770477	


In [24]:
distribution_1 = distribution_proba_WB(1, validation_2, 0)
print('Result on alpha smoothing 1grams', perplexity(distribution_1, validation_output))
distribution_2 = distribution_proba_WB(2, validation_2, 0)
print('Result on alpha smoothing 2grams', perplexity(distribution_2, validation_output))
distribution_3 = distribution_proba_WB(3, validation_3, 0)
print('Result on alpha smoothing 3grams', perplexity(distribution_3, validation_output))
distribution_4 = distribution_proba_WB(4, validation_4, 0)
print('Result on alpha smoothing 4grams', perplexity(distribution_4, validation_output))

1-grams.hdf5	


Result on alpha smoothing 1grams	6.023602585938	
1-grams.hdf5	


2-grams.hdf5	


Result on alpha smoothing 2grams	5.0433818281195	
1-grams.hdf5	


2-grams.hdf5	


...

Result on alpha smoothing 3grams	4.5689460403288	
1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


Result on alpha smoothing 4grams	4.4189330367685	


In [26]:
distribution_1 = distribution_proba_WB(1, validation_2, 1)
print('Result on alpha smoothing 1grams', perplexity(distribution_1, validation_output))
distribution_2 = distribution_proba_WB(2, validation_2, 1)
print('Result on alpha smoothing 2grams', perplexity(distribution_2, validation_output))
distribution_3 = distribution_proba_WB(3, validation_3, 1)
print('Result on alpha smoothing 3grams', perplexity(distribution_3, validation_output))
distribution_4 = distribution_proba_WB(4, validation_4, 1)
print('Result on alpha smoothing 4grams', perplexity(distribution_4, validation_output))

1-grams.hdf5	


Result on alpha smoothing 1grams	6.0029978880708	
1-grams.hdf5	


2-grams.hdf5	


Result on alpha smoothing 2grams	5.026210590868	
1-grams.hdf5	


2-grams.hdf5	


...

Result on alpha smoothing 3grams	4.5520551448742	
1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


Result on alpha smoothing 4grams	4.4020577683054	


In [28]:
distribution_5 = distribution_proba_WB(5, validation_5, 1)
print('Result on alpha smoothing 4grams', perplexity(distribution_5, validation_output))

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


5-grams.hdf5	


Result on alpha smoothing 4grams	4.3572276127565	


In [32]:
distribution_6 = distribution_proba_WB(6, validation_6, 1)
print('Result on alpha smoothing 4grams', perplexity(distribution_6, validation_output))

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


5-grams.hdf5	


6-grams.hdf5	


Result on alpha smoothing 4grams	4.3336779743836	


In [29]:
math.log(4.3572276127565)

1.4718359864769	


In [33]:
test_6:size()

 3761
   55
[torch.LongStorage of size 2]



In [34]:
-- Applying on test
distribution_test_6 = distribution_proba_WB(6, test_6, 1)

1-grams.hdf5	


2-grams.hdf5	


3-grams.hdf5	


4-grams.hdf5	


5-grams.hdf5	


6-grams.hdf5	


In [35]:
-- Saving the current best model
myFile = hdf5.open('pred_test_wb_6', 'w')
myFile:write('distribution', distribution_test_6)
myFile:close()

# Accuracy

In [12]:
m, true_output = validation_output:max(2)

In [13]:
function compute_accuracy(pred, true_output)
    max,argmax = pred:max(2)
    acc = 0
    for i = 1, true_output:size(1) do
        if argmax[i][1] == true_output[i][1] then
            acc = acc + 1
        end
    end
    score = acc/true_output:size(1)
    
    return score
end

In [14]:
print('Result on alpha smoothing ', compute_accuracy(distribution_2, true_output))


Result on alpha smoothing 	0.59614243323442	
